In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
df_userXproduct = pd.read_csv('data/userXproduct_train.csv',index_col=['user_id','product_id'],#nrows=10000,
                             dtype={'user_id':np.uint32,
                                    'product_id':np.uint32,
                                    'mean_atco':np.float32,
                                    'mean_order_how':np.float32,
                                    'number_reord':np.uint8,
                                    'dblo':np.float32,
                                    'dslo':np.uint8,
                                    'mean_order_hod':np.float32,
                                    'mean_order_dow':np.float32,
                                    'reord_per_aisle':np.uint32,
                                    'ord_per_aisle':np.uint32,
                                    'reord_per_depart':np.uint32,
                                    'ord_per_depart':np.uint32,
                                    'train_order_number':np.uint8,
                                    'train_dspo':np.float16,
                                    'reord_per_user':np.uint16,
                                    'reord_per_dow':np.uint32,
                                    'ord_per_dow':np.uint32, 
                                    'reord_per_hod':np.uint32,
                                    'ord_per_hod':np.uint32,
                                    'reord_per_how':np.uint32,
                                    'ord_per_how':np.uint32
                                   })

In [3]:
df_userXproduct.columns

Index([u'mean_atco', u'mean_order_hod', u'mean_order_how', u'number_reord',
       u'dslo', u'mean_order_dow', u'dblo', u'reord_per_aisle',
       u'ord_per_aisle', u'reord_per_depart', u'ord_per_depart',
       u'train_order_number', u'train_dspo', u'order_dow', u'reord_per_dow',
       u'ord_per_dow', u'order_hod', u'reord_per_hod', u'ord_per_hod',
       u'order_how', u'reord_per_how', u'ord_per_how'],
      dtype='object')

In [4]:
df_target  = pd.read_csv('data/target_train.csv',index_col=['user_id','product_id'])
df_userXproduct = df_userXproduct.join(df_target)
df_userXproduct.fillna(False,inplace=True)

In [5]:
df_userXproduct['reordered_ratio'] = df_userXproduct['number_reord'] /(df_userXproduct['train_order_number']-1.)

In [ ]:
df_userXproduct.columns.values

array(['mean_atco', 'mean_order_hod', 'mean_order_how', 'number_reord',
       'dslo', 'mean_order_dow', 'dblo', 'reord_per_aisle',
       'ord_per_aisle', 'reord_per_depart', 'ord_per_depart',
       'train_order_number', 'train_dspo', 'order_dow', 'reord_per_dow',
       'ord_per_dow', 'order_hod', 'reord_per_hod', 'ord_per_hod',
       'order_how', 'reord_per_how', 'ord_per_how', 'reordered',
       'reordered_ratio'], dtype=object)

In [ ]:
X = df_userXproduct.reset_index()[['user_id',
                                   'product_id',
                                   'mean_atco', 
                                   'mean_order_hod',
                                   'mean_order_how', 
                                   'number_reord',
                                   'dslo', 
                                   'mean_order_dow',
                                   'dblo', 
                                   'reord_per_aisle',
                                   'ord_per_aisle', 
                                   'reord_per_depart', 
                                   'ord_per_depart',
                                   'train_order_number',
                                   'train_dspo',
                                   'order_dow', 
                                   'reord_per_dow',
                                   'ord_per_dow', 
                                   'order_hod',
                                   'reord_per_hod',
                                   'ord_per_hod',
                                   'order_how', 
                                   'reord_per_how',
                                   'ord_per_how', 
                                   'reordered',
                                   'reordered_ratio'
                                    ]].values

y = df_userXproduct.reset_index()[['user_id','product_id','reordered']].values

print(X.shape)
print(y.shape)

In [ ]:
#cols = [                           'mean_atco', 
#                                   'mean_order_how', 
#                                   #'number_reord', 
#                                   #'mean_order_dow', 
#                                   'reord_per_aisle', 
#                                   'reord_per_depart',
#                                   #'train_order_number', 
#                                   'train_dspo', 
#                                   'reord_per_user',
#                                   #'reord_per_dow', 
#                                   #'reord_per_hod', 
#                                   'reord_per_how', 
#                                   'reordered_ratio'
#                                    ]

In [ ]:
#g = sns.pairplot(df_userXproduct.loc[:10,cols])

# Neural net

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# fix random seed for reproducibility
np.random.seed(7)
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.utils.np_utils import to_categorical

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import scale

In [ ]:
X[:,2:] = scale(X[:,2:],axis=0)

In [ ]:
from sklearn.model_selection import GroupKFold
groups_user_ids = X[:,0]
group_kfold = GroupKFold(n_splits=4)
ind_train, ind_test = next(group_kfold.split(X, y, groups_user_ids))

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.6,random_state=42)
X_train = X[ind_train]
y_train = y[ind_train]
X_test  = X[ind_test]
y_test  = y[ind_test]

print (X_train.shape,y_train.shape)
print (X_test.shape,y_test.shape)
print (float(len(X_test))/(len(X_test)+len(X_train)))

y_train = np.append(y_train,to_categorical(y_train[:,2]),axis=1)
#y_test = np.append(y_test,to_categorical(y_test[:,2]),axis=1)

In [ ]:
inds_pos = np.argwhere(y_train[:,2]==True).reshape(-1)
inds_neg = np.argwhere(y_train[:,2]==False).reshape(-1)

print(float(len(inds_pos))/(len(inds_pos)+len(inds_neg)))
inds_neg = np.random.choice(inds_neg,size=len(inds_pos))
inds = np.random.permutation(np.append(inds_pos,inds_neg))

print(float(len(inds_pos))/(len(inds)))

X_train = X_train[inds]
y_train = y_train[inds]

In [ ]:
def fit_model_keras():
    # create the model
    model = Sequential()
    model.add(Dense(100, activation='relu', input_shape = (X_train.shape[1]-2,)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])      
    
    print(model.summary())
    early_stopping_monitor = EarlyStopping(patience=2)
    model.fit(X_train[:,2:], y_train[:,[3,4]], epochs=20, batch_size=64,validation_split=0.3,callbacks = [early_stopping_monitor])


    from keras.models import load_model
    model.save('model_file2.h5')
    


In [ ]:
def fit_model_sklearn():
    from sklearn.linear_model import LogisticRegression
    from sklearn.svm import SVC
    from sklearn.neighbors import KNeighborsClassifier
    #y_train[:,4].reshape(-1).astype(int)
    y_prepro = y_train[:,4].reshape(-1).astype(np.int)

    classifier = LogisticRegression()
    #classifier = SVC(kernel="linear", C=0.025)
    #classifier = KNeighborsClassifier(3)
    classifier.fit(X_train[:,2:], y_prepro[:])

In [ ]:
import xgboost as xgb
def fit_model_xgb():
    
    # specify parameters via map
    num_round = 20
    
    xgb_params = {
        #'n_trees': 50, 
        'eta': 0.1,
        'max_depth': 12,
        'subsample': 0.76,
        'objective': 'binary:logistic', #'reg:logistic',#'binary:logistic'
        'eval_metric': ['logloss','auc'],
        'lambda': 10,
        'gamma': .7,
        'colsample_bytree':.95,
        'silent': 0,
        'alpha': 2e-05
    }

    
    y_prepro = y_train[:,4].reshape(-1).astype(np.int)
    dtrain = xgb.DMatrix(X_train[:,2:],label=y_prepro)
   
    
    return xgb.train(xgb_params, dtrain, num_round,verbose_eval=True)
    # make prediction
    

bst = fit_model_xgb()

dtest  = xgb.DMatrix(X_test[:,2:])
preds = bst.predict(dtest)

bst.save_model('model_xgb.xgb')

In [ ]:
df_resultats = pd.DataFrame(y_test,columns=['user_id','product_id','test'])
df_resultats['pred_false'] = 0. 
df_resultats['pred_true'] = preds

In [ ]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print "\nModel Report"
    print "Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions)
    print "AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob)
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
#Choose all predictors except target & IDcols
predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb1 = XGBClassifier(
        learning_rate =0.1,
         n_estimators=1000,
         max_depth=5,
         min_child_weight=1,
         gamma=0,
         subsample=0.8,
         colsample_bytree=0.8,
         objective= 'binary:logistic',
         nthread=4,
         scale_pos_weight=1,
         seed=27)
modelfit(xgb1, train, predictors)

In [ ]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
                                                   min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                                   objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                        param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
import cPickle as pickle
pickle.dump(bst, open("model_xgb.dat", "wb"))

In [ ]:
import xgboost as xgb
y_prepro = y_train[:,4].reshape(-1).astype(np.int)

#gbm = xgb.XGBClassifier(max_depth=3,
#                        n_estimators=300,
#                        learning_rate=0.05).fit(X_train[:,2:],y_prepro)

gbm = xgb.XGBClassifier(max_depth=6,
                        #n_estimators=300,
                        subsample=0.76,
                        objective='reg:logistic',
                        reg_lambda=10,
                        reg_alpha=2e-05,
                        gamma=.7,
                        colsample_bytree=.95,
                        silent=1,
                        learning_rate=0.1).fit(X_train[:,2:],y_prepro)

predictions = gbm.predict(X_test[:,2:])

In [ ]:
import cPickle as pickle
pickle.dump(gbm, open("model_xgb.dat", "wb"))

In [ ]:
y_pred = gbm.predict_proba(X_test[:,2:])
y_pred = np.append(y_test,y_pred,axis=1)

In [ ]:
y_pred.shape

# Prediction

In [ ]:
mod = 'keras'
    
if mod == 'keras':
    fit_model_keras()
    from keras.models import load_model
    model = load_model('model_file2.h5')

    y_pred = np.append(y_test,model.predict(X_test[:,2:]),axis=1)
    
else:
    fit_model_sklearn()
    
    y_pred = classifier.predict_proba(X_test[:,2:])
    y_pred = np.append(y_test,y_pred,axis=1)

In [ ]:
df_resultats = pd.DataFrame(y_pred,columns=['user_id','product_id','test','pred_false','pred_true'])
                        

In [ ]:
df_resultats = df_resultats.set_index(['user_id','product_id'])
df_resultats['test'] = df_resultats['test'].astype(np.bool)
#df_resultats['pred_false'] = df_resultats['pred_false'].astype(np.float32)
df_resultats['pred_true'] = df_resultats['pred_true'].astype(np.float32)
del df_resultats['pred_false']

In [ ]:
# Generate the confusion matrix and classification report
print(confusion_matrix(df_resultats.test, df_resultats.pred_true>0.5))
print(classification_report(df_resultats.test, df_resultats.pred_true>0.5))

In [ ]:
fpr, tpr, thresholds = roc_curve(df_resultats.test, df_resultats.pred_true)

# Plot ROC curve
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr,tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

In [ ]:
roc_score = roc_auc_score(df_resultats.test, df_resultats.pred_true)
print (roc_score)

In [ ]:
print ('Reading the orders dataset...')
df_orders = pd.read_csv('data/orders.csv',dtype={'order_id':np.uint32,
                                                 'user_id':np.uint32,
                                                 'order_number':np.uint8,
                                                 'order_dow':np.uint8,
                                                 'order_hour_of_day':np.uint8,
                                                 'days_since_prior_order':np.float16})

df_orders_test = df_orders.loc[df_orders['eval_set']=='train']

print('Reading the train products dataset...')
df_products_train = pd.read_csv('data/order_products__train.csv',dtype={'order_id':np.uint32,
                                                                        'product_id':np.uint32,
                                                                        'add_to_cart_order':np.uint8,
                                                                        'reordered':np.bool})

In [ ]:
df_resultats_true = df_resultats.loc[df_resultats.test==True,'test'].reset_index()
df_resultats_true = df_resultats_true.groupby(['user_id'])['product_id'].apply(lambda x : ' '.join(x.astype(str)))
df_resultats_true = df_resultats_true.rename('product_id_true')

user_id_to_predict = df_resultats.reset_index().user_id.unique()

In [ ]:
def test_pred(df_resultats,decision_threshold):

    #decision_threshold = 0.3
    df_resultats_pred = df_resultats.loc[df_resultats.pred_true>decision_threshold].reset_index()
    df_resultats_pred['pred_true'] = True
    df_resultats_pred = df_resultats_pred.groupby(['user_id'])['product_id'].apply(lambda x : ' '.join(x.astype(str)))
    df_resultats_pred = df_resultats_pred.rename('product_id')

    df_resultats_final = pd.concat([df_resultats_true,df_resultats_pred],axis=1)

    df_resultats_final = df_resultats_final.reindex(user_id_to_predict)
    df_resultats_final = df_resultats_final.fillna('None')

    df_resultats_final = df_orders_test.join(df_resultats_final,on='user_id',how='right')[['order_id',
                                                                                         'product_id_true',
                                                                                         'product_id']].sort_values('order_id') 
    
    df_resultats_final['f1'] = df_resultats_final.apply(f1_score_perso,axis=1)    
    return decision_threshold,np.mean(df_resultats_final['f1'])

In [ ]:
#from wikipedia : https://en.wikipedia.org/wiki/F1_score
# f1 = 2 * precision * recall / (precision+recall)
# precision  = number of true posisive  / number of predicted positive
# recall     = number of true posisive  / number of positives

def f1_score_perso(row):
    y_pred = row['product_id']
    y_true = row['product_id_true']
    if y_pred == 'None':
        y_pred = [0]
    else:
        y_pred = [int(y) for y in y_pred.split(' ')]
    if y_true == 'None':
        y_true = [0]
    else:
        y_true = [int(y) for y in y_true.split(' ')]
    true_pos = np.intersect1d(y_pred,y_true)
    precision = len(true_pos) / float(len(y_pred))
    recall = len(true_pos) / float(len(y_true))
    if precision+recall == 0:
        f1 = 0.
    else :
        f1 = 2 * precision * recall / (precision+recall)
    return f1

In [ ]:
#decision_thresholds = np.arange(0.1,0.95,0.1)
decision_thresholds = np.arange(0.4,0.76,0.01)
#decision_thresholds = np.array([0.5])
#decision_thresholds = np.arange(0.05,0.26,0.01)
#decision_thresholds = np.arange(0.6,1.,0.05)
#decision_thresholds = []
f1_sco = []

for decision_threshold in decision_thresholds:
    var_tmp = test_pred(df_resultats, decision_threshold)
    f1_sco.append(var_tmp[1])
    print (var_tmp)

    
ind_best = np.argmax(f1_sco)
print('=======================================================================')
print ('auroc:',roc_score, ',th:',decision_thresholds[ind_best],',f1:', f1_sco[ind_best])